In [1]:
import pandas as pd
import numpy as np
from scipy.linalg import inv
from PortOpt_factor.optimizer import pyport
from ipca_utils import impute_w_median, normalize, IPCA_factor


### Preprocess 

In [2]:
fn = "data/kelly_data_without_nanocap.p"
df = pd.read_pickle(fn)
cols_to_drop = ["isin", "cusip", "sedol", "excntry"]
df = df.drop(cols_to_drop, axis=1)
df = df.dropna(subset=['ret_local_lead1m'])
pd.set_option("display.max_column", None)
df.head(10)

,id,eom,ret_exc_lead1m,ret_local_lead1m,ret_local,ret_exc,niq_su,ret_6_1,ret_12_1,saleq_su,tax_gr1a,ni_inc8q,prc_highprc_252d,resff3_6_1,resff3_12_1,be_me,debt_me,at_me,ret_60_12,ni_me,fcf_me,div12m_me,eqpo_me,eqnpo_me,sale_gr3,sale_gr1,ebitda_mev,sale_me,ocf_me,ival_me,bev_mev,netdebt_me,eq_dur,capex_abn,at_gr1,ppeinv_gr1a,noa_at,noa_gr1a,lnoa_gr1a,capx_gr1,capx_gr2,capx_gr3,chcsho_12m,eqnpo_12m,debt_gr3,inv_gr1,inv_gr1a,oaccruals_at,taccruals_at,cowc_gr1a,coa_gr1a,col_gr1a,nncoa_gr1a,ncoa_gr1a,ncol_gr1a,nfna_gr1a,sti_gr1a,lti_gr1a,fnl_gr1a,be_gr1a,oaccruals_ni,taccruals_ni,netis_at,eqnetis_at,dbnetis_at,niq_be,niq_be_chg1,niq_at,niq_at_chg1,ebit_bev,ebit_sale,sale_bev,at_turnover,gp_at,gp_atl1,ope_be,ope_bel1,op_at,op_atl1,cop_at,cop_atl1,f_score,o_score,z_score,pi_nix,at_be,saleq_gr1,rd_me,rd_sale,opex_at,emp_gr1,rd5_at,age,dsale_dinv,dsale_drec,dgp_dsale,dsale_dsga,sale_emp_gr1,tangibility,kz_index,ocfq_saleq_std,cash_at,ni_ar1,ni_ivol,earnings_variability,aliq_at,aliq_mat,seas_1_1an,seas_1_1na,seas_2_5an,seas_2_5na,seas_6_10an,seas_6_10na,seas_11_15an,seas_11_15na,seas_16_20an,seas_16_20na,market_equity,ivol_ff3_21d,ivol_capm_252d,ivol_capm_21d,ivol_hxz4_21d,rvol_21d,beta_60m,betabab_1260d,beta_dimson_21d,turnover_126d,turnover_var_126d,dolvol_126d,dolvol_var_126d,prc,ami_126d,zero_trades_21d,zero_trades_126d,zero_trades_252d,rmax1_21d,rskew_21d,iskew_capm_21d,iskew_ff3_21d,iskew_hxz4_21d,coskew_21d,ret_1_0,betadown_252d,bidaskhl_21d,ret_3_1,ret_9_1,ret_12_7,corr_1260d,rmax5_21d,rmax5_rvol_21d,ni_be,ocf_at,ocf_at_chg1,mispricing_perf,mispricing_mgmt,qmj,qmj_prof,qmj_growth,qmj_safety
0,100100001,1970-09-30,0.059230,0.063830,NaN,NaN,NaN,NaN,NaN,NaN,0.055691,NaN,NaN,NaN,NaN,0.389696,0.333709,1.095773,NaN,0.077817,-0.613949,NaN,NaN,NaN,8.273810,4.052973,0.147433,1.399905,-0.425303,1.022609,0.525045,0.284973,16.828861,-0.021824,3.848362,2.787572,2.823370,2.191658,0.138015,1.048487,26.461111,40.889831,NaN,NaN,6.091646,10.403599,0.281903,0.452041,0.237322,0.312587,0.505851,0.193264,0.139454,0.222834,0.083380,-0.214719,NaN,0.032843,0.251010,0.266091,7.349377,3.858437,NaN,NaN,0.251010,0.060123,NaN,0.052671,NaN,0.222367,0.105129,2.115171,2.159265,0.425118,2.061128,0.434923,1.727343,0.172889,0.838230,-0.279152,-1.353428,6.0,-1.548939,NaN,2.143262,2.81187,0.335870,NaN,NaN,1.129423,NaN,NaN,117,-7.392944,-5.827022,0.736966,-0.389403,NaN,0.626638,NaN,NaN,0.044476,-0.604826,0.060335,0.222692,3.209009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.202500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.179708,-0.388130,-0.166415,NaN,NaN,NaN,NaN,NaN,NaN
1,100100001,1970-10-31,-0.284600,-0.280000,0.063830,0.059230,NaN,NaN,NaN,NaN,0.055691,NaN,NaN,NaN,NaN,0.366314,0.313686,1.030027,NaN,0.080072,-0.577112,NaN,NaN,NaN,8.273810,4.052973,0.140456,1.288395,-0.399785,0.961253,0.500198,0.267874,16.979129,-0.021824,3.848362,2.787572,2.823370,2.191658,0.138015,1.048487,26.461111,40.889831,NaN,NaN,6.091646,10.403599,0.281903,0.452041,0.237322,0.312587,0.505851,0.193264,0.139454,0.222834,0.083380,-0.214719,NaN,0.032843,0.251010,0.266091,7.349377,3.858437,NaN,NaN,0.251010,NaN,NaN,NaN,NaN,0.222367,0.105129,2.115171,2.159265,0.425118,2.061128,0.434923,1.727343,0.172889,0.838230,-0.279152,-1.353428,6.0,-1.548939,NaN,2.143262,2.81187,-0.071796,NaN,NaN,1.129423,NaN,NaN,118,-7.392944,-5.827022,0.736966,-0.389403,NaN,0.626638,NaN,NaN,0.044476,-0.604826,0.060335,0.222692,3.209009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.875000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.063830,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.179708,-0.388130,-0.166415,0.557225,0.007569,-0.131257,-0.242119,1.131001,-1.038514
2,100100001,1970-11-30,0.106911,0.111111,-0.280000,-0.284600,NaN,NaN,NaN,NaN,0.055691,NaN,NaN,NaN,NaN,0.508769,0.435675,1.430593,NaN,0.111211,-0.801545,NaN,NaN,NaN,8.273810,4.052973,0.180267,1.789437,-0.555257,1.335073,0.641973,0.3

In [3]:
df['ret_local_lead1m'].describe()

count    2.456676e+06
mean     1.276360e-02
std      2.156571e-01
min     -9.999000e-01
25%     -5.580880e-02
50%      4.215649e-03
75%      6.960005e-02
max      2.201268e+02
Name: ret_local_lead1m, dtype: float64

In [6]:
df[df['ret_local_lead1m']>1e02].sort_values(by='eom')

,id,eom,ret_exc_lead1m,ret_local_lead1m,ret_local,ret_exc,niq_su,ret_6_1,ret_12_1,saleq_su,tax_gr1a,ni_inc8q,prc_highprc_252d,resff3_6_1,resff3_12_1,be_me,debt_me,at_me,ret_60_12,ni_me,fcf_me,div12m_me,eqpo_me,eqnpo_me,sale_gr3,sale_gr1,ebitda_mev,sale_me,ocf_me,ival_me,bev_mev,netdebt_me,eq_dur,capex_abn,at_gr1,ppeinv_gr1a,noa_at,noa_gr1a,lnoa_gr1a,capx_gr1,capx_gr2,capx_gr3,chcsho_12m,eqnpo_12m,debt_gr3,inv_gr1,inv_gr1a,oaccruals_at,taccruals_at,cowc_gr1a,coa_gr1a,col_gr1a,nncoa_gr1a,ncoa_gr1a,ncol_gr1a,nfna_gr1a,sti_gr1a,lti_gr1a,fnl_gr1a,be_gr1a,oaccruals_ni,taccruals_ni,netis_at,eqnetis_at,dbnetis_at,niq_be,niq_be_chg1,niq_at,niq_at_chg1,ebit_bev,ebit_sale,sale_bev,at_turnover,gp_at,gp_atl1,ope_be,ope_bel1,op_at,op_atl1,cop_at,cop_atl1,f_score,o_score,z_score,pi_nix,at_be,saleq_gr1,rd_me,rd_sale,opex_at,emp_gr1,rd5_at,age,dsale_dinv,dsale_drec,dgp_dsale,dsale_dsga,sale_emp_gr1,tangibility,kz_index,ocfq_saleq_std,cash_at,ni_ar1,ni_ivol,earnings_variability,aliq_at,aliq_mat,seas_1_1an,seas_1_1na,seas_2_5an,seas_2_5na,seas_6_10an,seas_6_10na,seas_11_15an,seas_11_15na,seas_16_20an,seas_16_20na,market_equity,ivol_ff3_21d,ivol_capm_252d,ivol_capm_21d,ivol_hxz4_21d,rvol_21d,beta_60m,betabab_1260d,beta_dimson_21d,turnover_126d,turnover_var_126d,dolvol_126d,dolvol_var_126d,prc,ami_126d,zero_trades_21d,zero_trades_126d,zero_trades_252d,rmax1_21d,rskew_21d,iskew_capm_21d,iskew_ff3_21d,iskew_hxz4_21d,coskew_21d,ret_1_0,betadown_252d,bidaskhl_21d,ret_3_1,ret_9_1,ret_12_7,corr_1260d,rmax5_21d,rmax5_rvol_21d,ni_be,ocf_at,ocf_at_chg1,mispricing_perf,mispricing_mgmt,qmj,qmj_prof,qmj_growth,qmj_safety
2421513,118217201,2020-05-31,220.126661,220.126761,0.510638,0.510538,NaN,1.0,2.76,NaN,NaN,NaN,1.0,-0.031953,0.006671,NaN,NaN,NaN,-0.99989,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.003527,-0.003521,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,137,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.36,0.120622,-0.352989,7.594427,-0.325512,0.10874,NaN,NaN,NaN,NaN,40.40353,0.092407,0.079628,0.087585,0.09173,0.105424,3.325909,0.029655,6.315613,0.002175,1.934359,47061.749347,2.294204,0.0142,4.888075,0.003683,0.004917,0.004861,0.206186,0.29016,0.611692,0.576877,0.352831,0.168301,0.510638,1.062284,0.055976,1.238095,0.46875,1.0,0.007535,0.163989,2.010382,NaN,NaN,NaN,0.992366,NaN,NaN,NaN,NaN,1.73147


In [3]:
from datetime import date
problem_date = df[df['eom']==date(1993,9,30)]
problem_date.columns[problem_date.isna().all()]

Index([], dtype='object')

In [8]:
characteristics = df.columns[6:] #list of characteristics
df_ipca = impute_w_median(df, characteristics)
# df_ipca_normalized = normalize(df_ipca, characteristics)

In [9]:
pd.set_option('display.max_columns', None)
df_ipca.columns[df_ipca.isna().any(axis=0)]

Index(['ret_local', 'ret_exc', 'niq_su', 'ret_6_1', 'ret_12_1', 'saleq_su',
       'ni_inc8q', 'prc_highprc_252d', 'resff3_6_1', 'resff3_12_1',
       'ret_60_12', 'div12m_me', 'eqpo_me', 'eqnpo_me', 'chcsho_12m',
       'eqnpo_12m', 'sti_gr1a', 'netis_at', 'eqnetis_at', 'niq_be',
       'niq_be_chg1', 'niq_at', 'niq_at_chg1', 'z_score', 'saleq_gr1',
       'kz_index', 'ocfq_saleq_std', 'aliq_mat', 'seas_1_1an', 'seas_1_1na',
       'seas_2_5an', 'seas_2_5na', 'seas_6_10an', 'seas_6_10na',
       'seas_11_15an', 'seas_11_15na', 'seas_16_20an', 'seas_16_20na',
       'ivol_ff3_21d', 'ivol_capm_252d', 'ivol_capm_21d', 'ivol_hxz4_21d',
       'rvol_21d', 'beta_60m', 'betabab_1260d', 'beta_dimson_21d',
       'turnover_126d', 'turnover_var_126d', 'dolvol_126d', 'dolvol_var_126d',
       'ami_126d', 'zero_trades_21d', 'zero_trades_126d', 'zero_trades_252d',
       'rmax1_21d', 'rskew_21d', 'iskew_capm_21d', 'iskew_ff3_21d',
       'iskew_hxz4_21d', 'coskew_21d', 'ret_1_0', 'betadown_252d',


## IPCA Parameters

In [10]:
from logger import ErrorLogger
from tqdm import tqdm
import os
from datetime import datetime

In [11]:
window_size = 240 #rolling window size
K = 6 #num of principle components
unique_dates = sorted(df_ipca['eom'].unique()) #unique dates
T = len(unique_dates)

current_date = datetime.now().strftime('%Y-%m-%d')
# log_fp = "logs/"+f"{current_date}-w{window_size}-log-error.txt"
res_fp = "results/"+f"{current_date}-w{window_size}-results.csv"
logger = ErrorLogger() #each node/window has separate log file

print (f'There are {T} unique days starting with {unique_dates[0]}')
print (f'Total {len(characteristics)} charateristics starting with {characteristics[0]}')

There are 745 unique days starting with 1962-01-31
Total 153 charateristics starting with niq_su


In [9]:
pl_date = date(2003,10,31)
for t in range(window_size, T):
    if unique_dates[t] == pl_date:
        print (t)
        break  

501


In [6]:
np.random.seed(42)

t = 501
K = 6
    
window_dates = unique_dates[t-window_size:t]
window_data = df_ipca[df_ipca['eom'].isin(window_dates)]
date_to_predict = unique_dates[t]

print (f'======Progress: {t}: {date_to_predict}======')

# calculate ipca
try:
    Gamma, Factors, r_t, X_last = IPCA_factor(window_data, characteristics, K)
except Exception as e:
    raise

# regularization
V_t = inv(Gamma.T @ X_last.T @ X_last @ Gamma) @ Gamma.T @ X_last.T
reg_mat = np.zeros_like(V_t)
reg_mat[:K, :K] = np.eye(K)*1e-03
V_t += reg_mat

Sigma_t= np.cov(Factors, rowvar = True)
mu_t = np.array(np.mean(Factors, axis=1))

# Grid search? grid search for regularization terms
g1 = np.exp(np.linspace( np.log(1e-6),np.log(5),10))
g2 = np.exp(np.linspace( np.log(1e-6),np.log(5),10))

OptimalPortfolioWeights_t = pyport.portfolio_optimization(
    meanVec=np.array(mu_t),
    sigMat=np.array(Sigma_t),
    retTarget=0,
    longShort=0.2,
    maxAlloc=0.08, #
    lambda_l1=g1[0],
    lambda_l2=g2[0],
    riskfree=0,
    assetsOrder=None,
    maxShar=1,
    factor=np.array(V_t.T)[:,0:K+1],
    turnover=None,
    w_pre=None,
    individual=False,
    exposure_constrain=0,
    w_bench=None,
    factor_exposure_constrain=None,
    U_factor=None,
    general_linear_constrain=None,
    U_genlinear=0,
    w_general=None,
    TE_constrain=0,
    general_quad=0,
    Q_w=None,
    Q_b=None,
    Q_bench=None
)[0].reshape(-1, 1)

ret_t = (V_t @ r_t).T @ OptimalPortfolioWeights_t

w_individual = np.dot(np.array(V_t.T), OptimalPortfolioWeights_t).flatten()


print (f'======Max_return: {t}: {ret_t}======')      
           
    

======Progress: 501: 2003-10-31======


The panel dimensions are:
n_samples: 9480 , L: 153 , T: 200


[========================================================================] 100%


Step 1 - Aggregate Update: 5724699465464.285
Step 2 - Aggregate Update: 19.005621997481494
Step 3 - Aggregate Update: 21.35241346721036
Step 4 - Aggregate Update: 13.075688345900232
Step 5 - Aggregate Update: 8.627405420676414
Step 6 - Aggregate Update: 7.2378677712955
Step 7 - Aggregate Update: 4.459751027708633
Step 8 - Aggregate Update: 3.500314088172334
Step 9 - Aggregate Update: 2.791258482679534
Step 10 - Aggregate Update: 0.9003102435676418
Step 11 - Aggregate Update: 2.179409661964046
Step 12 - Aggregate Update: 1.3964593064402324
Step 13 - Aggregate Update: 1.8492295794670652
Step 14 - Aggregate Update: 1.0896629596609726
Step 15 - Aggregate Update: 2.0685136679989546
Step 16 - Aggregate Update: 4.408694566908543
Step 17 - Aggregate Update: 2.869720732244929
Step 18 - Aggregate Update: 5.029039473393876
Step 19 - Aggregate Update: 5.188577223628582
Step 20 - Aggregate Update: 4.057214352759431
Step 21 - Aggregate Update: 3.9940078825350493
Step 22 - Aggregate Update: 2.5802839

KeyboardInterrupt: 

## Multiprocessing

In [11]:
import multiprocessing
import os

In [12]:
os.cpu_count() 

128

In [9]:
num_cores = multiprocessing.cpu_count()
num_cores

128